In [20]:
from langchain_openai import ChatOpenAI
from langchain.schema import HumanMessage, SystemMessage
from typing import List, Optional
from pydantic import BaseModel, Field
from langchain.schema import HumanMessage, SystemMessage

In [28]:
class IsSmellPresent(BaseModel):
    isSmellPresent: bool

class SmellReasoningAndRefactoredTestCode(BaseModel):
    reasonWhySmellExists: str
    refactoredTestCode: str

class Smell(BaseModel):
    smellName: str
    smellDescription: str
    detectionTechnique: str

In [ ]:
llm = ChatOpenAI(
    model="gpt-4o",
    temperature=0,
    api_key="OPENAI_API_KEY",
)

structured_llm = llm.with_structured_output(IsSmellPresent)
structured_llm_v2 = llm.with_structured_output(SmellReasoningAndRefactoredTestCode)

In [9]:
smellList = [
    Smell(smellName="Assertion Roulette", 
          smellDescription="Occurs when a test method has multiple non-documented assertions. Multiple assertion statements in a test method without a descriptive message impacts readability/understandability/maintainability as it’s not possible to understand the reason for the failure of the test.", 
          detectionTechnique="A test method contains more than one assertion statement without without an explanation/message (parameter in the assertion method)."),
    Smell(smellName="Conditional Test Logic",
          smellDescription="Test methods need to be simple and execute all statements in the production method. Conditions within the test method will alter the behavior of the test and its expected output, and would lead to situations where the test fails to detect defects in the production method since test statements were not executed as a condition was not met. Furthermore, conditional code within a test method negatively impacts the ease of comprehension by developers.",
          detectionTechnique="A test method that contains one or more control statements (i.e if, switch, conditional expression, for, foreach and while statement)."),
    Smell(smellName="Constructor Initialization",
          smellDescription="Ideally, the test suite should not have a constructor. Initialization of fields should be in the setUp() method. Developers who are unaware of the purpose of setUp() method would give rise to this smell by defining a constructor for the test suite.",
          detectionTechnique="A test class that contains a constructor declaration."),
    Smell(smellName="Default Test",
          smellDescription="By default Android Studio creates default test classes when a project is created. These classes are meant to serve as an example for developers when wring unit tests and should either be removed or renamed. Having such files in the project will cause developers to start adding test methods into these files, making the default test class a container of all test cases. This also would possibly cause problems when the classes need to be renamed in the future.",
          detectionTechnique="A test class is named either `ExampleUnitTest' or `ExampleInstrumentedTest'."),
    Smell(smellName="Duplicate Assert",
          smellDescription="This smell occurs when a test method tests for the same condition multiple times within the same test method. If the test method needs to test the same condition using different values, a new test method should be utilized; the name of the test method should be an indication of the test being performed. Possible situations that would give rise to this smell include: (1) developers grouping multiple conditions to test a single method; (2) developers performing debugging activities; and (3) an accidental copy-paste of code.",
          detectionTechnique="A test method that contains more than one assertion statement with the same parameters."),
    Smell(smellName="Eager Test",
          smellDescription="Occurs when a test method invokes several methods of the production object. This smell results in difficulties in test comprehension and maintenance.",
          detectionTechnique="A test method contains multiple calls to multiple production methods."),
    Smell(smellName="Empty Test",
          smellDescription="Occurs when a test method does not contain executable statements. Such methods are possibly created for debugging purposes and then forgotten about or contains commented out code. An empty test can be considered problematic and more dangerous than not having a test case at all since JUnit will indicate that the test passes even if there are no executable statements present in the method body. As such, developers introducing behavior-breaking changes into production class, will not be notified of the alternated outcomes as JUnit will report the test as passing.",
          detectionTechnique="A test method that does not contain a single executable statement."),
    Smell(smellName="Exception Handling",
          smellDescription="This smell occurs when a test method explicitly a passing or failing of a test method is dependent on the production method throwing an exception. Developers should utilize JUnit's exception handling to automatically pass/fail the test instead of writing custom exception handling code or throwing an exception.",
          detectionTechnique="A test method that contains either a throw statement or a catch clause."),
    Smell(smellName="General Fixture",
          smellDescription="Occurs when a test case fixture is too general and the test methods only access part of it. A test setup/fixture method that initializes fields that are not accessed by test methods indicates that the fixture is too generalized. A drawback of it being too general is that unnecessary work is being done when a test method is run.",
          detectionTechnique="Not all fields instantiated within the setUp method of a test class are utilized by all test methods in the same test class."),
    Smell(smellName="Ignored Test",
          smellDescription="JUnit 4 provides developers with the ability to suppress test methods from running. However, these ignored test methods result in overhead since they add unnecessary overhead with regards to compilation time, and increases code complexity and comprehension.",
          detectionTechnique="A test method or class that contains the @Ignore annotation."),
    Smell(smellName="Lazy Test",
          smellDescription="Occurs when multiple test methods invoke the same method of the production object.",
          detectionTechnique="Multiple test methods calling the same production method."),
    Smell(smellName="Magic Number Test",
          smellDescription="Occurs when assert statements in a test method contain numeric literals (i.e., magic numbers) as parameters. Magic numbers do not indicate the meaning/purpose of the number. Hence, they should be replaced with constants or variables, thereby providing a descriptive name for the input.",
          detectionTechnique="An assertion method that contains a numeric literal as an argument."),
    Smell(smellName="Mystery Guest",
          smellDescription="Occurs when a test method utilizes external resources (e.g. files, database, etc.). Use of external resources in test methods will result in stability and performance issues. Developers should use mock objects in place of external resources.",
          detectionTechnique="A test method containing object instances of files and databases classes."),
    Smell(smellName="Redundant Print",
          smellDescription="Print statements in unit tests are redundant as unit tests are executed as part of an automated process with little to no human intervention. Print statements are possibly used by developers for traceability and debugging purposes and then forgotten.",
          detectionTechnique="A test method that invokes either the print or println or printf or write method of the System class."),
    Smell(smellName="Redundant Assertion",
          smellDescription="This smell occurs when test methods contain assertion statements that are either always true or always false. This smell is introduced by developers for debugging purposes and then forgotten.",
          detectionTechnique="A test method that contains an assertion statement in which the expected and actual parameters are the same."),
    Smell(smellName="Resource Optimism",
          smellDescription="This smell occurs when a test method makes an optimistic assumption that the external resource (e.g., File), utilized by the test method, exists.",
          detectionTechnique="A test method utilizes an instance of a File class without calling the exists(), isFile() or notExists() methods of the object."),
    Smell(smellName="Sensitive Equality",
          smellDescription="Occurs when the toString method is used within a test method. Test methods verify objects by invoking the default toString() method of the object and comparing the output against an specific string. Changes to the implementation of toString() might result in failure. The correct approach is to implement a custom method within the object to perform this comparison.",
          detectionTechnique="A test method invokes the toString() method of an object."),
    Smell(smellName="Sleepy Test",
          smellDescription="Explicitly causing a thread to sleep can lead to unexpected results as the processing time for a task can differ on different devices. Developers introduce this smell when they need to pause execution of statements in a test method for a certain duration (i.e. simulate an external event) and then continuing with execution.",
          detectionTechnique="A test method that invokes the Thread.sleep() method."),
    Smell(smellName="Unknown Test",
          smellDescription="An assertion statement is used to declare an expected boolean condition for a test method. By examining the assertion statement it is possible to understand the purpose of the test method. However, It is possible for a test method to written sans an assertion statement, in such an instance JUnit will show the test method as passing if the statements within the test method did not result in an exception, when executed. New developers to the project will find it difficult in understanding the purpose of such test methods (more so if the name of the test method is not descriptive enough).",
          detectionTechnique="A test method that does not contain a single assertion statement and @Test(expected) annotation parameter.")
]

In [16]:
def getSmell(smellName: str) -> Smell:
    for smell in smellList:
        if smell.smellName == smellName:
            return smell
    return None

def getSmellDescription(smell: Smell) -> str:    
    description = f"""
    The name of the test code smell is {smell.smellName}
    Description of {smell.smellName} is : {smell.smellDescription}
    It can be detected using: {smell.detectionTechnique}
    """
    return description

In [ ]:
def detectSmell(testCode: str, productionCode: str, smell: Smell) -> bool:
    
    smellDetails = getSmellDescription(smell)
    
    message = [
        SystemMessage(
            "You are an expert in detecting one test smells. You will be given a test code block and a production code block. You have to identify if that code blocks contain the following test smell:" + smellDetails
        ),
        HumanMessage("Here is the test code block:" + testCode + "Here is the production code block:" + productionCode + "Can you tell which test smell(s) the test code block contains"),
    ]
    
    response = structured_llm.invoke(message)    
    
    return response.isSmellPresent

In [30]:
def refactorAndExplainSmell(testCode: str, productionCode: str, smell: Smell) -> SmellReasoningAndRefactoredTestCode:
    
    smellDetails = getSmellDescription(smell)
    
    message = [
        SystemMessage(
            "You are an expert in explaining test smell reasoning and refactoring. You will be given a test code block and a production code block. The test code contains the following test smell:" + smellDetails + " You have to explain why the code contains this smell and also give a refactored version so that the code does not contains the test smell"
        ),
        HumanMessage("Here is the test code block:" + testCode + "Here is the production code block:" + productionCode + "Can you tell why the test smell exists and the refactored test code?"),
    ]
    
    response = structured_llm_v2.invoke(message)
    
    return response

In [45]:
def smellAgent(testCode: str, productionCode: str, smellName: str):
    smell = getSmell(smellName)
        
    if(type(smell) == type(None)):
        print("THE SMELL " + str(smellName) + " DOES NOT EXISTS")
        return
    
    isSmellPresent = detectSmell(testCode, productionCode, smell)
    
    if isSmellPresent == True:
        res = refactorAndExplainSmell(testCode, productionCode, smell)
        print("THE TEST CODE CONTAINS " + smell.smellName)
        print("\n\nREASON: " + res.reasonWhySmellExists)
        print("\n\nREFACTORED VERSION : " + res.refactoredTestCode)
    else:
        print("THE TEST CODE DO NOT CONTAIN " + smell.smellName)
    
    return

In [38]:
test_code = """
    @Test
    public void realCase() {
        Point p34 = new Point("34", 556506.667, 172513.91, 620.34, true);
        Point p45 = new Point("45", 556495.16, 172493.912, 623.37, true);
        Point p47 = new Point("47", 556612.21, 172489.274, 0.0, true);
        Abriss a = new Abriss(p34, false);
        a.removeDAO(CalculationsDataSource.getInstance());
        a.getMeasures().add(new Measure(p45, 0.0, 91.6892, 23.277, 1.63));
        a.getMeasures().add(new Measure(p47, 281.3521, 100.0471, 108.384, 1.63));

        try {
            a.compute();
        } catch (CalculationException e) {
            Assert.fail(e.getMessage());
        }

        // test intermediate values with point 45
        Assert.assertEquals("233.2405",
            this.df4.format(a.getResults().get(0).getUnknownOrientation()));
        Assert.assertEquals("233.2435",
            this.df4.format(a.getResults().get(0).getOrientedDirection()));
        Assert.assertEquals("-0.1", this.df1.format(
            a.getResults().get(0).getErrTrans()));

        // test intermediate values with point 47
        Assert.assertEquals("233.2466",
            this.df4.format(a.getResults().get(1).getUnknownOrientation()));
        Assert.assertEquals("114.5956",
            this.df4.format(a.getResults().get(1).getOrientedDirection()));
        Assert.assertEquals("0.5", this.df1.format(
            a.getResults().get(1).getErrTrans()));

        // test final results
        Assert.assertEquals("233.2435", this.df4.format(a.getMean()));
        Assert.assertEquals("43", this.df0.format(a.getMSE()));
        Assert.assertEquals("30", this.df0.format(a.getMeanErrComp()));
    }
    """
production_code = "not available"

smellAgent(test_code, production_code, "Assertion Roulette")

THE TEST CODE CONTAINS Assertion Roulette


REASON: The test method `realCase` contains multiple assertions without any descriptive messages. This makes it difficult to understand which specific assertion failed and why, especially when the test fails. Each assertion checks a different aspect of the computation, but without messages, it's not clear what each assertion is verifying. This lack of clarity can make the test harder to maintain and debug, as developers need to manually trace back the failure to understand the context.


REFACTORED VERSION : @Test
public void realCase() {
    Point p34 = new Point("34", 556506.667, 172513.91, 620.34, true);
    Point p45 = new Point("45", 556495.16, 172493.912, 623.37, true);
    Point p47 = new Point("47", 556612.21, 172489.274, 0.0, true);
    Abriss a = new Abriss(p34, false);
    a.removeDAO(CalculationsDataSource.getInstance());
    a.getMeasures().add(new Measure(p45, 0.0, 91.6892, 23.277, 1.63));
    a.getMeasures().add(new Measure(p47

In [47]:
smellAgent(test_code, production_code, "Assertion Roulettee")

THE SMELL Assertion Roulettee DOES NOT EXISTS


In [48]:
smellAgent(test_code, production_code, "Sleepy Test")

THE TEST CODE DO NOT CONTAIN Sleepy Test
